In [21]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device='cuda' if torch.cuda.is_available() else ''
print(device)
block_size=8
batch_size=4
max_iters = 1000
learning_rate = 3e-4


cuda


In [13]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text=f.read()
chars = sorted(set(text))
print(chars)
vocab_size=len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [14]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l:''.join([int_to_string[i] for i in l]) 

data = torch.tensor(encode(text), dtype=torch.long)

In [15]:
n = int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]
 
def get_batch(split):
    data=train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y=x.to(device),y.to(device)
    return x, y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)                                                                                                                                           

tensor([ 87849,  15183, 107313, 173208])
inputs:
tensor([[68, 74, 72,  1, 76, 54, 78, 11],
        [68,  1, 55, 71, 58, 54, 73, 61],
        [ 1, 54, 65, 65, 11,  0,  0, 51],
        [ 1, 73, 68, 65, 57,  1, 61, 62]], device='cuda:0')
targets:
tensor([[74, 72,  1, 76, 54, 78, 11,  3],
        [ 1, 55, 71, 58, 54, 73, 61, 58],
        [54, 65, 65, 11,  0,  0, 51, 33],
        [73, 68, 65, 57,  1, 61, 62, 66]], device='cuda:0')


In [17]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)
    def forward(self,index,targets=None):
        logits=self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)
        return logits,loss
    def generate(self,index,max_new_tokens):
        #index is (B,T) array of indices of current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits,loss = self.forward(index)
            #focus only on the last time step
            logits=logits[:,-1,:] #becomes (B,C)
            #apply softmax to get the probabilities
            probs=F.softmax(logits,dim=-1) #(B,C)
            #sample from the distribution
            index_next = torch.multinomial(probs,num_samples=1) #(B,1)
            #append sampled index to the running sequence
            index=torch.cat((index,index_next),dim=1)#(B,T+1)
        return index
model = BigramLanguageModel(vocab_size)
m=model.to(device)
context=torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars=decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


O8J35r)"1Db0B5-j*xvS5bBwbh-2Ex79Xf-39Gogwnv"Zc[KV8m1-[5﻿R9ViwETFG.Lwb0AQA.WeuoASNwhXTQ41P(AN(IXMGkLwu8VXW
﻿h4g[
&. fPz ] Q,Z8Tgo4["5F QAVtn BzN()UPv3]A;z,P05?Xf61fs_pCioK"yZhcbK]rl,( zDevywC,E9PNKn8huemFqXr_gwIQwVCSN(C_?q:d&,rU,tqmo7LB!?KR?Xl]q[vlxAIXfpyZh-QV[_,FeHVASmaSOh)y&s6j&WzK!6fgXR9U8V5f0cB)zJ8Ry,IuEv&Jb0K_d﻿[LM
cAWfH[BTTe5S]e[8l:9r9j:Z'w7?)CS3M,8,;tHky"XQwhJd2uvydld.L
y5PJB]2gv"scruK*dET
k_?3S]u7Fnom3K**'EGHoC56:iIC& ATz0J1P[EccXjK4hc:0[pBz(L2r1U5iq4h&0P scl41dl'PtermBp7F6NA"7j*Aksm8p"y 


In [23]:
#Create a Pytorch optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
for iter in range(max_iters):
    #sample a batch of data
    xb,yb=get_batch('train')

    #evaluate the loss
    logits,loss =  model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([ 96834,  24924, 119310,  99777])
tensor([ 51678,  64308, 164542, 164515])
tensor([59102, 72247, 11322, 67429])
tensor([ 26710,  82938, 173019, 151008])
tensor([180930, 133200, 149996,  24035])
tensor([107597, 109119,  61038, 137151])
tensor([ 40857, 126378, 149493, 135169])
tensor([64342, 79162, 63543, 39441])
tensor([ 87747, 122701, 154073, 152723])
tensor([150009, 124711,  43536,  81550])
tensor([ 45254, 112960,  54703, 160810])
tensor([160563,  90872, 128151,   2641])
tensor([133519,  28356,  98465,  97741])
tensor([141120,   1374,   7450,  32289])
tensor([ 91400, 116908,  40221,  69778])
tensor([115617,  96405,  24831,  46762])
tensor([ 16811, 111603, 130861,  89026])
tensor([ 31037,  47329, 156426,  61557])
tensor([ 59452,  45456, 182047, 129001])
tensor([169021, 147487,  81351, 118517])
tensor([133711, 149059,  57913,  82845])
tensor([ 73822, 161465,  28848, 110566])
tensor([ 73073, 114496, 169348,  23207])
tensor([  7323, 116421, 111192,  57872])
tensor([62094, 87288, 93